In [ ]:
TODO:
#Use player scores to inform priors
#Use first few weeks to assess "market confidence"
#Finish Cumulative Season Stat Generation
#Other Measures of defensive prowess?
#Use Previous Years of Data

#Incorporate data from:
#- Sequential Home/Away
#- Player Scores
#- Injuries
#- Time Zones
#- Weather

Currently tests two baseline models for the 2017 season.  Will update with more data as it is acquired.

First baseline is an average of the pointspread predictions from various casinos and betting outlets. This does not require training, though we could train a ridge regression model and expect a similar result (as we'd expect the betting outlets to be relatively equally predictive on average as well as fairly correlated, so their weights should be very close under L2-regularized linear regression).

In [3]:
import numpy as np
import pandas as pd

In [12]:
odds = pd.read_csv("base_data/NFL_Odds.csv")

#Formatting to create key and filter
odds["datestring"] = pd.to_datetime(odds['date']).dt.strftime('%Y%m%d')
odds["Spread_key"] = odds['datestring'] + odds['Home']
odds['Spread_val'] = odds['HomeScore'] - odds['AwayScore']
spreads = odds.filter(regex = "Spread_")

#Compute square loss over 2017 season
X = spreads.values[:,:-2].astype('float32')
y = spreads['Spread_val'].values.astype('float32')
scores = np.nanmean(X, axis = 1)
residual = (scores - y)**2
loss = np.nanmean(residual)
print(loss)

336.946


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


In [15]:
#Import basic game stats for each NFL game from 2013-2017 seasons

season_frames = []
for y in range(2013,2018):
    temp = pd.read_csv("base_data/nflstats" + str(y) + ".csv")
    temp.rename(index = str, columns = {'HPS': 'APS', 'HPSY': 'APSY', 'HPS.1':'HPS', 'HPSY.1':'HPSY'}, inplace = True)
    season_frames.append(temp)

game_stats = pd.concat(season_frames, ignore_index = True)
game_stats['Datetime'] = pd.to_datetime(game_stats['Start'])
game_stats['Key'] = game_stats['Datetime'].dt.strftime('%Y%m%d') + game_stats['Home']
game_stats['Spread'] = game_stats['HPts'] - game_stats['APts']


In [27]:
game_stats.describe()

,Season,Week,APts,HPts,OverUnder,VegasLine,AFD,AFum,AFumL,APY,...,HFum,HFumL,HPY,HPA,HPI,HPS,HPSY,HRA,HRY,Spread
count,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,...,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000,1335.000000
mean,2015.000000,9.469663,21.431461,23.962547,45.268539,-2.278652,19.404494,1.238202,0.570787,231.814981,...,1.322846,0.591760,241.922097,34.974532,0.849438,2.261423,14.598502,27.340075,114.024719,2.531086
std,1.414744,5.291256,9.593596,10.336546,4.010058,5.556568,4.869941,1.132275,0.769090,74.380154,...,1.195899,0.767846,76.599740,8.434559,0.955038,1.703195,12.501478,7.469458,50.168055,14.308249
min,2013.000000,1.000000,0.000000,0.000000,36.000000,-26.500000,6.000000,0.000000,0.000000,6.000000,...,0.000000,0.000000,46.000000,7.000000,0.000000,0.000000,0.000000,9.000000,7.000000,-38.000000
25%,2014.000000,5.000000,14.000000,17.000000,42.500000,-6.500000,16.000000,0.000000,0.000000,178.000000,...,0.000000,0.000000,188.000000,29.000000,0.000000,1.000000,6.000000,22.000000,80.000000,-6.000000
50%,2015.000000,10.000000,21.000000,24.000000,45.000000,-3.000000,20.000000,1.000000,0.000000,227.000000,...,1.000000,0.000000,239.000000,35.000000,1.000000,2.000000,12.000000,27.000000,109.000000,3.000000
75%,2016.000000,14.000000,27.000000,30.000000,48.000000,2.500000,23.000000,2.000000,1.000000,283.000000,...,2.000000,1.000000,293.000000,41.000000,1.000000,3.000000,21.000000,33.000000,142.000000,11.000000
max,2017.000000,21.000000,56.000000,57.000000,59.500000,14.500000,37.000000,7.000000,5.000000,498.000000,...,7.000000,3.000000,522.000000,70.000000,5.000000,10.000000,72.000000,55.000000,355.000000,52.000000


In [17]:
def weekly_team_stats(stats, team):
    cols = ['Key','Pts','FD','Fum','FumL','PY','PA','PI','PS', 'PSY', 'RA', 'RY']
    t_stats = stats.query('Home == @team | Away == @team')
    weekly_stats = pd.DataFrame(columns = cols)
    count = 0
    
    for index,week in t_stats.iterrows():
        week = week.to_dict()
        
        if team == week['Home']:
            prefix = 'H'
            week.pop('Home')
        else:
            prefix = 'A'
            week.pop('Away')
        
        stats = {k[1:]: v for k, v in week.items() if k[0] == prefix}
        stats['Key'] = week['Key']
        weekly_stats = weekly_stats.append(stats, ignore_index = True)
        
    return weekly_stats

In [ ]:
def avg_team_stats(weekly_stats):
    cols = ['Key','C_Pts','C_FD','C_Fum','C_FumL','C_PY','C_PA','C_PI','C_PS', 'C_PSY', 'C_RA', 'C_RY']
    avg_stats = weekly_stats[]
    for index,week in 

In [20]:
teams = game_stats['Home'].unique()
team_stats = {}
for t in teams:
    team_stats.update({t:weekly_team_stats(game_stats, t)})
    


    


In [25]:
print(team_stats['LAR'])

            Key Pts  FD Fum FumL   PY  PA PI PS PSY  RA   RY
0   20160912SFO   0  10   0    0  120  35  2  2  10  23   65
1   20160918LAR   9  17   0    0  219  30  0  3  20  25   64
2   20160925TAM  37  18   1    1  183  26  1  1   7  33  137
3   20161002ARI  17  12   1    1  247  30  0  2  19  23   41
4   20161009LAR  19  23   2    1  243  31  2  4  28  29  102
5   20161016DET  28  23   1    0  311  32  1  1  10  22   76
6   20161023LAR  10  20   4    0  271  53  4  3  20  20   74
7   20161106LAR  10  18   1    0  280  46  1  4  16  17   59
8   20161113NYJ   9  16   1    0  155  30  0  3  10  34  125
9   20161120LAR  10  12   1    1  125  31  0  1   9  28  102
10  20161127NOR  21  13   1    1  178  32  1  3  36  17   69
11  20161204NWE  10   7   0    0  126  32  2  4  35  13   36
12  20161211LAR  14  21   3    3  208  41  2  3  27  23  104
13  20161215SEA   3  13   1    0  136  35  0  4  31  17   47
14  20161224LAR  21  13   2    0   78  24  2  4  12  29   99
15  20170101LAR   6   9 